In [1]:
!pip install scikit-optimize
!pip install skorch

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import joblib as jb
import numpy as np
import pandas as pd
import os
import sys
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import skopt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import average_precision_score
from sklearn.base import clone
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
import sklearn
from skorch import NeuralNetBinaryClassifier
import skorch
import torch
import torch.nn as nn
from torch import optim
import itertools
from joblib import Parallel, delayed

# Package versions
print("Python version:", sys.version)
print("Joblib version:", jb.__version__)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("scikit-optimize version:", skopt.__version__)
print("scikit-learn version:", sklearn.__version__)
print("torch version:", torch.__version__)
print("skorch version:", skorch.__version__)

Python version: 3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0]
Joblib version: 1.4.2
NumPy version: 1.26.4
Pandas version: 2.2.3
scikit-optimize version: 0.10.2
scikit-learn version: 1.4.1.post1
torch version: 2.3.1+cu121
skorch version: 1.1.0


In [3]:
data_per_year = jb.load("data_per_year.pkl")
data_per_year[2015].head()

,sex,healthcare_plan,procedure,age,no_show,distance_to_clinic,waiting_time,day_of_week,month,hour,is_holiday_tomorrow,was_holiday_yesterday,no_show_history,no_show_rate_last_1_day_in_MR,median_no_show_rate_last_7_days_in_MR,median_no_show_rate_last_30_days_in_MR,quantile25_no_show_rate_last_7_days_in_MR,quantile25_no_show_rate_last_30_days_in_MR,quantile75_no_show_rate_last_7_days_in_MR,quantile75_no_show_rate_last_30_days_in_MR,mean_no_show_rate_last_7_days_in_MR,mean_no_show_rate_last_30_days_in_MR,min_no_show_rate_last_7_days_in_MR,min_no_show_rate_last_30_days_in_MR,max_no_show_rate_last_7_days_in_MR,max_no_show_rate_last_30_days_in_MR,std_no_show_rate_last_7_days_in_MR,std_no_show_rate_last_30_days_in_MR,no_show_rate_last_1_day_clinic_wide,median_no_show_rate_last_7_days_clinic_wide,median_no_show_rate_last_30_days_clinic_wide,quantile25_no_show_rate_last_7_days_clinic_wide,quantile25_no_show_rate_last_30_days_clinic_wide,quantile75_no_show_rate_last_7_days_clinic_wide,quantile75_no_show_rate_last_30_days_clinic_wide,mean_no_show_rate_last_7_days_clinic_wide,mean_no_show_rate_last_30_days_clinic_wide,min_no_show_rate_last_7_days_clinic_wide,min_no_show_rate_last_30_days_clinic_wide,max_no_show_rate_last_7_days_clinic_wide,max_no_show_rate_last_30_days_clinic_wide,std_no_show_rate_last_7_days_clinic_wide,std_no_show_rate_last_30_days_clinic_wide,today_demand_MR,demand_last_1_day_MR,median_demand_last_7_days_MR,median_demand_last_30_days_MR,quantile25_demand_last_7_days_MR,quantile25_demand_last_30_days_MR,quantile75_demand_last_7_days_MR,quantile75_demand_last_30_days_MR,mean_demand_last_7_days_MR,mean_demand_last_30_days_MR,min_demand_last_7_days_MR,min_demand_last_30_days_MR,max_demand_last_7_days_MR,max_demand_last_30_days_MR,std_demand_last_7_days_MR,std_demand_last_30_days_MR,today_demand_clinic_wide,demand_last_1_day_clinic_wide,median_demand_last_7_days_clinic_wide,median_demand_last_30_days_clinic_wide,quantile25_demand_last_7_days_clinic_wide,quantile25_demand_last_30_days_clinic_wide,quantile75_demand_last_7_days_clinic_wide,quantile75_demand_last_30_days_clinic_wide,mean_demand_last_7_days_clinic_wide,mean_demand_last_30_days_clinic_wide,min_demand_last_7_days_clinic_wide,min_demand_last_30_days_clinic_wide,max_demand_last_7_days_clinic_wide,max_demand_last_30_days_clinic_wide,std_demand_last_7_days_clinic_wide,std_demand_last_30_days_clinic_wide
0,F,Ipasgo,Cervical Spine MRI,69.0,0.0,4.668,15.0,4,1,9,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0.089112,0.236111,0.176419,0.191913,0.135563,0.1,0.019231,0.297872,0.297872,0.064484,0.067978,11.0,2.0,3.0,8.0,2.0,4.25,7.0,11.0,4.714286,7.7,1.0,1.0,11.0,18.0,3.76883,4.510358,30.0,18.0,30.0,73.5,23.0,34.25,47.0,85.75,37.0,64.466667,16.0,16.0,73.0,109.0,18.624101,28.996245
1,F,Ipasgo,Knee MRI,50.0,0.0,NaN,18.0,4,1,19,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0.089112,0.236111,0.176419,0.191913,0.135563,0.1,0.019231,0.297872,0.297872,0.064484,0.067978,11.0,2.0,3.0,8.0,2.0,4.25,7.0,11.0,4.714286,7.7,1.0,1.0,11.0,18.0,3.76883,4.510358,30.0,18.0,30.0,73.5,23.0,34.25,47.0,85.75,37.0,64.466667,16.0,16.0,73.0,109.0,18.624101,28.996245
2,F,Hapvida,Knee MRI,39.0,0.0,13.090,18.0,4,1,19,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0.089112,0.236111,0.176419,0.191913,0.135563,0.1,0.019231,0.297872,0.297872,0.064484,0.067978,11.0,2.0,3.0,8.0,2.0,4.25,7.0,11.0,4.714286,7.7,1.0,1.0,11.0,18.0,3.76883,4.510358,30.0,18.0,30.0,73.5,23.0,34.25,47.0,85.75,37.0,64.466667,16.0,16.0,73.0,109.0,18.624101,28.996245
3,F,Ipasgo,Knee MRI,17.0,0.0,75.092,18.0,4,1,19,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.046875,0.095238,0.060628,0.0,0.0,0.666667,0.666667,0.233285,0.143218,0.222222,0.178082,0.127717,0.147606,0

In [4]:
# Preprocessing pipeline for the 'sex' column
sex_encoding = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("one_hot", OneHotEncoder(sparse_output=False, drop="first")) # Drop first to avoid collinearity
])

# Preprocessing step for other categorical variables
other_cat = ['healthcare_plan', 'procedure', 'day_of_week', 'month', 'hour']
one_hot_other_cat = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")  # Drop first to avoid collinearity

# Preprocessing step for historic data from the clinic
# 1) No-show statistics
no_show_rate_cols = ['no_show_rate_last_1_day_in_MR',
       'median_no_show_rate_last_7_days_in_MR',
       'median_no_show_rate_last_30_days_in_MR',
       'quantile25_no_show_rate_last_7_days_in_MR',
       'quantile25_no_show_rate_last_30_days_in_MR',
       'quantile75_no_show_rate_last_7_days_in_MR',
       'quantile75_no_show_rate_last_30_days_in_MR',
       'mean_no_show_rate_last_7_days_in_MR',
       'mean_no_show_rate_last_30_days_in_MR',
       'min_no_show_rate_last_7_days_in_MR',
       'min_no_show_rate_last_30_days_in_MR',
       'max_no_show_rate_last_7_days_in_MR',
       'max_no_show_rate_last_30_days_in_MR',
       'std_no_show_rate_last_7_days_in_MR',
       'std_no_show_rate_last_30_days_in_MR',
       'no_show_rate_last_1_day_clinic_wide',
       'median_no_show_rate_last_7_days_clinic_wide',
       'median_no_show_rate_last_30_days_clinic_wide',
       'quantile25_no_show_rate_last_7_days_clinic_wide',
       'quantile25_no_show_rate_last_30_days_clinic_wide',
       'quantile75_no_show_rate_last_7_days_clinic_wide',
       'quantile75_no_show_rate_last_30_days_clinic_wide',
       'mean_no_show_rate_last_7_days_clinic_wide',
       'mean_no_show_rate_last_30_days_clinic_wide',
       'min_no_show_rate_last_7_days_clinic_wide',
       'min_no_show_rate_last_30_days_clinic_wide',
       'max_no_show_rate_last_7_days_clinic_wide',
       'max_no_show_rate_last_30_days_clinic_wide',
       'std_no_show_rate_last_7_days_clinic_wide',
       'std_no_show_rate_last_30_days_clinic_wide']

# 2) Demand statistics
demand_cols = ['demand_last_1_day_MR', 'median_demand_last_7_days_MR',
       'median_demand_last_30_days_MR', 'quantile25_demand_last_7_days_MR',
       'quantile25_demand_last_30_days_MR', 'quantile75_demand_last_7_days_MR',
       'quantile75_demand_last_30_days_MR', 'mean_demand_last_7_days_MR',
       'mean_demand_last_30_days_MR', 'min_demand_last_7_days_MR',
       'min_demand_last_30_days_MR', 'max_demand_last_7_days_MR',
       'max_demand_last_30_days_MR', 'std_demand_last_7_days_MR',
       'std_demand_last_30_days_MR', 'demand_last_1_day_clinic_wide',
       'median_demand_last_7_days_clinic_wide',
       'median_demand_last_30_days_clinic_wide',
       'quantile25_demand_last_7_days_clinic_wide',
       'quantile25_demand_last_30_days_clinic_wide',
       'quantile75_demand_last_7_days_clinic_wide',
       'quantile75_demand_last_30_days_clinic_wide',
       'mean_demand_last_7_days_clinic_wide',
       'mean_demand_last_30_days_clinic_wide',
       'min_demand_last_7_days_clinic_wide',
       'min_demand_last_30_days_clinic_wide',
       'max_demand_last_7_days_clinic_wide',
       'max_demand_last_30_days_clinic_wide',
       'std_demand_last_7_days_clinic_wide',
       'std_demand_last_30_days_clinic_wide']

# Corresponding PCA objects
pca_no_show = PCA(whiten=True, n_components=0.95)
pca_demand  = PCA(whiten=True, n_components=0.95)

# Preparing transformers list
transformers = [
    ("sex", sex_encoding, ["sex"]),
    ("OHE", one_hot_other_cat, other_cat),
    ("PCA_no_show", pca_no_show, no_show_rate_cols),
    ("PCA_demand", pca_demand, demand_cols),
    ("numerical_no_indicator", Pipeline([
        ("standardize", StandardScaler()),
        ("impute", SimpleImputer(strategy="median")) # simple_imputer without missing indicator
    ]), ["age", "distance_to_clinic"]),
    ("numerical", Pipeline([
        ("standardize", StandardScaler()),
        ("impute", SimpleImputer(strategy="median", add_indicator=True)) # simple_imputer with missing indicator
    ]), ["waiting_time", "no_show_history", "today_demand_clinic_wide", "today_demand_MR"])
]

# Initializing ColumnTransformer
col_transformer = ColumnTransformer(transformers, remainder="passthrough")

In [5]:
# MLP module
class MLP_module(nn.Module):
    def __init__(self, input_dim, n_hidden_dims, neurons_per_hidden, output_dim, dropout_rate=0.0):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        layers = []
        # Input layer
        layers.append(nn.Linear(input_dim, neurons_per_hidden))
        layers.append(nn.ReLU())
        # Hidden layers
        for _ in range(n_hidden_dims - 1):
            layers.append(nn.Linear(neurons_per_hidden, neurons_per_hidden))
            layers.append(nn.ReLU())
        # Output layer
        layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Linear(neurons_per_hidden, output_dim))
        self.layers = nn.Sequential(*layers)

    def forward(self, x, **kwargs):
        return self.layers(x)

# NeuralNetBinaryClassifier custom wrapper, so we can tune easily using scikit-optimize
class AnyInputNeuralNetBinaryClassifier(NeuralNetBinaryClassifier):
    def __init__(self, *args, pos_weight=1.0, **kwargs):
        if 'criterion__pos_weight' not in kwargs:
            kwargs['criterion__pos_weight'] = torch.tensor(pos_weight, dtype=torch.float)
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight

    def set_params(self, **params):
        if 'pos_weight' in params:
            pw = params.pop('pos_weight')
            params['criterion__pos_weight'] = torch.tensor(pw, dtype=torch.float)
            self.pos_weight = pw
        return super().set_params(**params)

    def fit(self, X, y, **fit_params):
        # Adjusting dtypes
        X32 = X.astype(np.float32)
        y32 = y.astype(np.float32)
        self.set_params(module__input_dim=X32.shape[1])
        return super().fit(X32, y32, **fit_params)

    def predict_proba(self, X, **kwargs):
        # Adjusting dtypes
        X32 = X.astype(np.float32)
        return super().predict_proba(X32, **kwargs)

    def predict(self, X, **kwargs):
        # Adjusting dtypes
        X32 = X.astype(np.float32)
        return super().predict(X32, **kwargs)


# Classifier
clf = AnyInputNeuralNetBinaryClassifier( # The majority of the values here are just dummy values,
    module=MLP_module,                   # as they will be set during hyperparameter tuning
    module__input_dim=1,
    module__output_dim=1,
    module__n_hidden_dims=2,
    module__neurons_per_hidden=128,
    module__dropout_rate=0.2,
    criterion=nn.BCEWithLogitsLoss,
    optimizer=optim.Adam,
    lr=1e-3,
    optimizer__weight_decay=0.0,
    max_epochs=500,
    batch_size=256,
    train_split=None,
    iterator_train__shuffle=True,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=0,
    pos_weight=1.0,
)


# Search space for hyperparameters
space  = [
    Integer(1, 4, name="module__n_hidden_dims"),
    Integer(64, 256, name="module__neurons_per_hidden"),
    Real(1e-5, 1e-1, prior="log-uniform", name="lr"),
    Real(1e-5, 10, prior="log-uniform", name="optimizer__weight_decay"),
    Real(0.0, 0.5, name="module__dropout_rate"),
    Real(1.0,  50.0, name="pos_weight")
]

# Final pipeline, including classifier
model_pipe = Pipeline([
    ("col_transformer", col_transformer),
    ("clf", clf)
])

In [6]:
def get_train_test(data_dict, test_year, initial_year):
    # Build training DataFrame from initial_year up to year-before test_year
    train_years = list(range(initial_year, test_year))
    df_train = pd.concat([data_dict[y] for y in train_years], ignore_index=True)
    df_test  = data_dict[test_year]

    X_train = df_train.drop(columns=["no_show"])
    y_train = df_train["no_show"].values
    X_test  = df_test.drop(columns=["no_show"])
    y_test  = df_test["no_show"].values

    # Return the number of training folds as weight
    return X_train, y_train, X_test, y_test, len(train_years)

class LoggingObjective:
    def __init__(self, data_per_year, test_years, model_pipe, search_space):
        self.data_per_year = data_per_year
        self.test_years = test_years
        self.initial_year = min(data_per_year.keys())
        self.model_pipe = model_pipe
        self.space = search_space
        self.log = []

    def evaluate_for_year(self, test_year, params):
        # Get train/test split + weight
        X_tr, y_tr, X_te, y_te, weight = get_train_test(
            self.data_per_year, test_year, self.initial_year
        )

        # Clone pipeline and set all hyperparameters on 'clf'
        model = clone(self.model_pipe)
        for dim, val in zip(self.space, params):
            model.set_params(**{f"clf__{dim.name}": val})

        # Fit and predict probabilities
        model.fit(X_tr, y_tr)
        y_proba = model.predict_proba(X_te)[:, 1]
        score   = average_precision_score(y_te, y_proba)
        return score, weight

    def __call__(self, x):
        # Parallel evaluation across years
        results = Parallel(n_jobs=-1)(
            delayed(self.evaluate_for_year)(yr, x) for yr in self.test_years
        )
        scores, weights = zip(*results)

        # Compute weighted mean & std
        mean = np.average(scores, weights=weights)
        std  = np.sqrt(np.average((np.array(scores)-mean)**2, weights=weights))

        # Log iteration details
        entry = {
            'params': {dim.name: val for dim, val in zip(self.space, x)},
            'scores': scores,
            'weights': weights,
            'weighted_mean': mean,
            'weighted_std': std
        }
        self.log.append(entry)

        # gp_minimize minimizes, so return negative mean
        return -mean

def hyperparameter_tuning(data_per_year,
                        test_years = [2016,2017,2018,2019,2020,2021,2022],
                        n_calls = 25,
                        n_initial_points = 10,
                        log_dir = "logs"):
    # Initialize objective and run gp_minimize
    objective = LoggingObjective(data_per_year, test_years, model_pipe, space)
    result = gp_minimize(
        objective,
        space,
        n_calls = n_calls,
        n_initial_points = n_initial_points,
        n_jobs = -1,
        verbose = True
    )

    # Extract and print best result
    best_score  = -result.fun
    best_params = {dim.name: val for dim, val in zip(space, result.x)}
    print(f"Best weighted average precision: {best_score:.4f}")
    print("Best hyperparameters:", best_params)

    # Save detailed log to disk
    os.makedirs(log_dir, exist_ok=True)
    with open(os.path.join(log_dir, "log.txt"), "w") as f:
        for i, e in enumerate(objective.log, 1):
            f.write(f"Iteration {i}\n")
            f.write(f"Params: {e['params']}\n")
            f.write(f"Scores: {e['scores']}\n")
            f.write(f"Weights: {e['weights']}\n")
            f.write(f"Weighted Mean: {e['weighted_mean']:.4f}\n")
            f.write(f"Weighted Std:  {e['weighted_std']:.4f}\n\n")

    return result, objective.log

In [7]:
result, objective_log = hyperparameter_tuning(data_per_year, n_calls=300, n_initial_points=100)
jb.dump(result, f"gp_minimize.pkl")
jb.dump(objective_log, f"objective_log.pkl")

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 128.0447
Function value obtained: -0.1157
Current minimum: -0.1157
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 122.7085
Function value obtained: -0.1090
Current minimum: -0.1157
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 133.1260
Function value obtained: -0.1179
Current minimum: -0.1179
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 111.6127
Function value obtained: -0.1470
Current minimum: -0.1470
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 136.8031
Function value obtained: -0.1120
Current minimum: -0.1470
Iteration No: 6

['objective_log.pkl']